In [2]:
!pip install ipykernel

In [4]:
!python -m ipykernel install --user --name=GPU_ENV_NAME

Installed kernelspec GPU_ENV_NAME in C:\Users\ashiv\AppData\Roaming\jupyter\kernels\gpu_env_name


In [5]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.test.is_gpu_available())

TensorFlow version: 2.12.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU available: False


In [ ]:
! nvidia-smi

In [ ]:
!git clone https://github.com/openai/shap-e

In [ ]:
%cd shap-e

In [ ]:
!pip install -e .

In [ ]:
!pip install shap-e torch torchvision flask


In [ ]:
import torch
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget, decode_latent_mesh
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shap_e.util.notebooks import decode_latent_mesh



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

In [ ]:
# Set the path to your data directory
data_path = "C:/Users/ashiv/Downloads/shap-e/samples"

In [ ]:
# Set the image dimensions and batch size
image_width = 64
image_height = 64
batch_size = 32

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Create an ImageDataGenerator to preprocess the GIF images
data_generator = ImageDataGenerator(rescale=1.0 / 255.0)

In [ ]:
# Get class names
class_names = sorted(set([os.path.basename(os.path.dirname(file)) for file in os.listdir(data_path) if file.endswith(".gif")]))

In [ ]:
# Generate training data from GIFs in different folders
train_generator = data_generator.flow_from_directory(
    data_path,
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

In [ ]:
# Get the class names from the generator
class_names = list(train_generator.class_indices.keys())

In [ ]:
# Print the different classes
print("Class Names:")
for class_name in class_names:
    print(class_name)

In [ ]:
batch_size = 1
guidance_scale = 15.0

prompt = input("Enter a prompt: ")

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

render_mode = 'nerf' # you can change this to 'stf'
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)

In [ ]:
def predict_label(images):
    # Get the label from the user
    label = input("Enter the label: ")

    # Placeholder code for prediction based on the rendered 3D model
    # You can replace this code with your own prediction algorithm

    # Return the label provided by the user
    return label

In [ ]:
# Example usage
images = [...]  # Provide the images of the rendered 3D model
predicted_label = predict_label(images)
print(predicted_label)

In [28]:
# Ground truth labels and predicted labels
ground_truth_labels = []
predicted_labels = []

for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

    # Get the ground truth label
    ground_truth_label = input(f"Enter the ground truth label for image {i+1}: ")
    ground_truth_labels.extend([ground_truth_label] * len(images))

    # Placeholder for your code to predict the label of the rendered 3D model
    predicted_label = predict_label(images)  # Modify this line with your actual prediction code
    predicted_labels.extend([predicted_label] * len(images))

KeyboardInterrupt: 

In [ ]:
# Calculate confusion matrix
labels = np.unique(ground_truth_labels)
confusion_mat = confusion_matrix(ground_truth_labels, predicted_labels, labels=labels)

In [ ]:
# Plot confusion matrix
fig, ax = plt.subplots()
im = ax.imshow(confusion_mat, interpolation='nearest', cmap=plt.cm.Blues)
ax.figure.colorbar(im, ax=ax)
ax.set(xticks=np.arange(len(labels)),
       yticks=np.arange(len(labels)),
       xticklabels=labels, yticklabels=labels,
       xlabel='Predicted label', ylabel='Ground truth label',
       title='Confusion Matrix')
plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
plt.show()

In [ ]:
# Calculate accuracy
accuracy = np.sum(np.diag(confusion_mat)) / np.sum(confusion_mat)

In [ ]:
# Print results
print("Confusion Matrix:")
print(confusion_mat)
print("Accuracy:", accuracy)

In [ ]:
# Plot correlation graph
plt.scatter(ground_truth_labels, predicted_labels)
plt.xlabel('Ground Truth')
plt.ylabel('Predicted')
plt.show()

In [ ]:
for i, latent in enumerate(latents):
    file_name = input(f"Enter a name for mesh {i+1}: ")
    with open(f'{file_name}.ply', 'wb') as f:
        decode_latent_mesh(xm, latent).tri_mesh().write_ply(f)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
